# RNN 실습

앞서 살펴본 예제를 바탕으로, 실제로 코드를 작성하는 실습을 진행합니다.

## 데이터 준비

필요한 라이브러리를 import하고, Sarcasm 데이터셋을 로드합니다.

Sarcasm 데이터셋은 뉴스 헤드라인이 모인 데이터로,

그 뉴스가 Sarcastic한, 즉 비꼬는 기사인지 분류하도록 되어있는 데이터입니다.

데이터를 로드하고, 파싱해줍니다.

<br>

그 후, 각종 parameter를 정의하고 데이터를 분리해줍니다.

In [ ]:
import numpy as np

import json
import tensorflow as tf

# 다운로드
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

# 파싱
sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

vocab_size = 1000 # 단어사전의 크기
embedding_dim = 16 # 토큰 임베딩 차원 수
max_length = 120 # 입력 시퀀스의 최대 길이를 지정
trunc_type='post' # max_length보다 긴 시퀀스의 경우 어디를 잘라낼 것인지 지정. pre는 앞쪽을, post는 뒷쪽을 잘라냄
padding_type='post' # max_length보다 긴 시퀀스의 경우 어디에 padding을 붙일 것인지 지정. pre는 앞쪽에, post는 뒷쪽에 붙임
oov_tok = "<OOV>" # Out Of Vocabulary 토큰
training_size = 20000 # trainings_size만 train 데이터로 사용, 나머지는 test 데이터로 사용

train_sentences = sentences[0:training_size]
test_sentences = sentences[training_size:]
train_labels = labels[0:training_size]
test_labels = labels[training_size:]

## **MISSION: 데이터 전처리**

##**예시 답안**

먼저, Tokenizer를 선언하고 학습 데이터를 정수 시퀀스로 변환합니다.

변환된 데이터의 길이를 맞추어 사용할 수 있게 조정합니다.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 크기가 vocab_size인 단어사전을 만듦
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)

# 위에서 만든 단어사전을 바탕으로 단어들을 정수 인덱스로 변환
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

# pad_sequences 함수를 통해 padding 혹은 truncating하여 모든 데이터의 길이를 max_length로 통일
# 길이가 짧은 데이터는 padding을 추가하고, 길이가 긴 데이터는 trunc_type 설정에 따라 잘라냄
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# numpy array 형태로 변환
train_padded = np.array(train_padded)
test_padded = np.array(test_padded)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## **MISSION: 네트워크 정의 및 학습**

##**예시 답안**

먼저, Embedding Layer를 추가합니다.

이 Embedding Layer는 단어사전의 크기, 임베딩 차원 수, 입력 시퀀스의 길이를 argument로 받습니다.

LSTM Layer를 Bidirectional하게 추가합니다.

추가적으로 RNN Layer를 쌓기 위해서 return_sequences=True 옵션을 추가합니다.

이번엔 GRU Layer를 Bidirectional하게 추가합니다.

RNN Layer와 Output Layer 사이에 적당한 크기의 Dense Layer를 추가합니다.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

## 모델 학습

위에서 전처리한 데이터와 정의한 네트워크로 모델을 학습합니다.

Validation accuracy가 80% 이상 나오도록 위의 네트워크를 설계해보세요

In [ ]:
num_epochs = 10
model.fit(train_padded, train_labels, epochs=num_epochs, validation_data=(test_padded, test_labels), verbose=1, batch_size=128)